# RDM Property and Analytics API Tutorial # 1

This notebook will leverage the REIT Data Market (RDM) Property API to collect, process and visualze Kilroy Realty (KRC) property locations.

## Notebook Requirements

- Python >= 3.9 (although older versions may work)
- Data Science Libraries
- San Francisco Counties GeoJSON

### Downloading San Francisco Counties GeoJSON

Execute the below to download the required data. Optionally, naviagte to the URL in a web browser.

In [ ]:
SF_NEIGHBORHOODS = "./sf_neighborhoods.geojson"

In [ ]:
!curl "https://data.sfgov.org/api/geospatial/pty2-tcw4?method=export&format=GeoJSON" --output $SF_NEIGHBORHOODS

In [ ]:
import os
import textwrap

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

Load the API key, create Rapid API headers, and set a base URL to use for queries.

In [ ]:
API_KEY = os.environ["RDM_RAPID_API_KEY"]

BASE_URL = "https://reit-property-data1.p.rapidapi.com"

HEADERS = {
    "X-RapidAPI-Key": API_KEY,
    "X-RapidAPI-Host": "reit-property-data1.p.rapidapi.com",
}

## GET /reit-index

Search the RDM Universe to idenitfy the number of quarters of data available for each REIT.


In [ ]:
url = f"{BASE_URL}/reit-index"

response = requests.request("GET", url, headers=HEADERS)
data = response.json()

In [ ]:
print(f'Number of REITs in RDM universe across all quarters is {len(data["reits"])}')

Loop through the list of tickers and create a Heat Map of of the RDM Universe

In [ ]:
sum_quarters = [len(data["reits"][ticker].keys()) for ticker in data["reits"]]
df = pd.DataFrame({"REIT": data["reits"].keys(), "TotalQuarters": sum_quarters})

df.head()

Create a Horizontal bar chart of RDM Property Data Holdings

In [ ]:
df.sort_values(by="TotalQuarters", ascending=True, inplace=True)

fig, ax = plt.subplots(figsize=(13, 40))
ax.barh(df["REIT"], df["TotalQuarters"], align="center")

ax.set_xlabel("Number of Quarters with Property Data")
ax.set_ylabel("REIT Ticker")
ax.set_title("Inspection of RDM Property API Holdings")
ax.tick_params(axis="both", which="major", labelsize=16)

# Make x-axis with minor ticks that are multiples
# of 1 and label minor ticks with '% 1.2f' formatting
plt.xticks(np.arange(min(df["TotalQuarters"]) - 1, max(df["TotalQuarters"]) + 1, 1.0))
ax.xaxis.set_major_formatter(FormatStrFormatter("%.0f"))
ax.margins(x=0)
plt.margins(y=0)

plt.show()

## GET /property 

Find properties in for Kilroy Realty (KRC) in Q4 2022. This method returns a list of property IDs.

In [ ]:
# Get property IDs for Kilroy Realty for properties owned in Q4 2022

url = f"{BASE_URL}/property"

querystring = {"reit": "krc", "qy": "Q42022"}

response = requests.request("GET", url, headers=HEADERS, params=querystring)

results = response.json()
property_ids = results["property_ids"]
property_ids[:5]  # Show the first 5

## GET /property-basic-info

Let's see ane example response from the `/property-basic-info` endpoint.

In [ ]:
url = f"{BASE_URL}/property-info-basic"
querystring = {"property_id": property_ids[0]}
response = requests.request("GET", url, headers=HEADERS, params=querystring)
response.json()

Now loop through property ids and acquire basic property information for each property owned by KRC.

In [ ]:
property_data = []
for property_id in property_ids:
    url = f"{BASE_URL}/property-info-basic"

    querystring = {"property_id": str(property_id)}

    response = requests.request("GET", url, headers=HEADERS, params=querystring)
    property_data.append(response.json())

Make a DataFrame with the results.

In [ ]:
df = pd.DataFrame(property_data)
df.head()

### Visualize the number of properties per MSA by Type

[This medium blog](https://medium.com/dunder-data/automatically-wrap-graph-labels-in-matplotlib-and-seaborn-a48740bc9ce) was used to wrap labels with the seaborn library.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
top10 = df["msa"].value_counts().index[:10]
ax.set_title("Total KRC Properties per MSA by Property Type - Q4 2022")
sns.countplot(
    data=df,
    x="msa",
    hue="property_type",
    order=top10,
    hue_order=["Office", "Retail", "HealthCare", "Residential"],
)
ax.legend(bbox_to_anchor=(1, 1), title="Property Type")


def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(
            textwrap.fill(text, width=width, break_long_words=break_long_words)
        )
    ax.set_xticklabels(labels, rotation=0)


# wrap the labels
wrap_labels(ax, 10)

ax.figure
ax.set_xlabel("Metropolitan Statistical Area (MSA)")
ax.set_ylabel("Number of Properties")
plt.show()

## GET /property-info-premium

Query premium attributes for KRC properties like latitude and longitude.

In [ ]:
location_info = []
for property_id in property_ids:
    url = f"{BASE_URL}/property-info-premium"

    querystring = {"property_id": property_id, "attribute": "location"}

    response = requests.request("GET", url, headers=HEADERS, params=querystring)
    location_info.append(response.json())

Make a DataFrame with the results.

In [ ]:
location_df = pd.json_normalize(location_info)
location_df.columns = ["id", "lat", "lon"]
location_df.head()

Join location data to other attributes.

In [ ]:
df = df.merge(location_df, on="id", how="left")
df.head()

### Plot Office Properties Located in the City of San Francisco on a Map

Use the geopandas library to load and read polygon file of San Francisco neighborhoods.

In [ ]:
sf_neighborhoods = gpd.read_file(SF_NEIGHBORHOODS)
sf_neighborhoods = sf_neighborhoods.to_crs("EPSG:4326")

# Filter to only San Francisco properties
sfprops = df.loc[(df["city"] == "San Francisco") & (df["property_type"] == "Office")]
sf_gpd = gpd.GeoDataFrame(
    sfprops, geometry=gpd.points_from_xy(sfprops.lon, sfprops.lat), crs="EPSG:4326"
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 25))

sf_neighborhoods_cx = sf_neighborhoods.to_crs(epsg="3857", inplace=False)
ax = sf_neighborhoods_cx.plot(edgecolor="grey", ax=ax, legend=False, alpha=0.25)
sf_gpd.to_crs(epsg="3857", inplace=True)
ax1 = sf_gpd.plot(ax=ax, color="red", markersize=40)


# Add basemap
cx.add_basemap(ax1, source=cx.providers.Stamen.TonerLite, zoom=13)
cx.add_basemap(ax1, source=cx.providers.Stamen.TonerLabels, zoom=13)

# Style the map
plt.tick_params(
    left=False, right=False, labelleft=False, labelbottom=False, bottom=False
)

plt.title(
    "Q4 2022 Kilroy Realty (KRC) - City of San Francisco - Office Properties", size=16
)

plt.show()

For help with this tutorial or for any other questions regarding the RDM Property and Analytics API, reach out to analytics@reitdatamarket.com